In [48]:
import warnings; warnings.filterwarnings('ignore')
import json, requests, pandas as pd, numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime
import pytz; tz_br = pytz.timezone('Brazil/East')

### Get google service credentials

In [49]:
project_id = 'pluvia-360323'
google_credentials = service_account.Credentials.from_service_account_file('../../../../Apps/Python/bolsao-api/credentials/pluvia-360323-eae2907a9c98.json')
client = bigquery.Client(credentials=google_credentials)

### Get metadata

In [50]:
# ...

### Custom bigquery functions

In [69]:
def groupby_first_query(db, col_id, col_date, col_time, select=['*'], max_delay=3):
    lookback = datetime.now() - max_delay * pd.offsets.Hour() # Hours offset partition to query
    return f'''
        WITH added_row_number AS (
          SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY {col_id} ORDER BY {col_date} DESC, {col_time} DESC) AS row_number
          FROM `{db}`
          WHERE {col_date} >= "{lookback.date()}" AND {col_time} >= "{str(lookback.time())[:8]}"
        )
        SELECT
          {', '.join(select)}
        FROM added_row_number
        WHERE row_number = 1
    '''

---
## Alerta-Rio API

In [9]:
from modules.alertario import alertario

#### Example usage

In [10]:
alerts = alertario.alert()

alerts.head()

,kind,read_at,name,is_new,location,acumulado_chuva_5_min,acumulado_chuva_15_min,acumulado_chuva_1_mes,acumulado_chuva_96_h,acumulado_chuva_24_h,acumulado_chuva_3_h,acumulado_chuva_2_h,acumulado_chuva_1_h,acumulado_chuva_4_h,id_estacao
0,pluviometric,2022-12-26T03:20:00-03:00,Vidigal,True,"[-22.9925, -43.233056]",0.0,0.0,119.6,1.8,0.0,0.0,0.0,0.0,0.0,1
1,pluviometric,2022-12-26T03:20:00-03:00,Urca,True,"[-22.955833, -43.166667]",0.0,0.0,109.6,3.8,0.2,0.0,0.0,0.0,0.0,2
2,pluviometric,2022-12-26T03:20:00-03:00,Rocinha,True,"[-22.985833, -43.245]",0.0,0.0,170.0,8.0,0.0,0.0,0.0,0.0,0.0,3
3,pluviometric,2022-12-26T03:20:00-03:00,Tijuca,True,"[-22.931944, -43.221667]",0.0,0.0,277.0,5.0,1.2,0.0,0.0,0.0,0.0,4
4,pluviometric,2022-12-26T03:20:00-03:00,Santa Teresa,True,"[-22.931667, -43.196389]",0.0,0.0,189.4,2.4,0.0,0.0,0.0,0.0,0.0,5


---
## Alerta-Rio bigquery dataset

In [13]:
alertario_db = 'datario.clima_pluviometro.taxa_precipitacao_alertario'

alertario_live_query = groupby_first_query(alertario_db, 'id_estacao', 'data_particao', 'horario', select=['*'], max_delay=3)
query_job = client.query(alertario_live_query)
alertario_live = pd.DataFrame(list(map(dict, query_job.result())))

alertario_live.head()

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,row_number
0,21_2022-12-26 03:00:00,21,0.0,0.0,0.0,0.0,14.2,03:00:00,2022-12-26,1
1,29_2022-12-26 03:00:00,29,0.0,0.0,0.0,0.0,8.0,03:00:00,2022-12-26,1
2,13_2022-12-26 03:00:00,13,0.0,0.0,0.0,0.0,10.0,03:00:00,2022-12-26,1
3,22_2022-12-26 03:00:00,22,0.0,0.0,0.0,0.0,5.2,03:00:00,2022-12-26,1
4,23_2022-12-26 03:00:00,23,0.0,0.0,0.0,0.0,2.2,03:00:00,2022-12-26,1


---
## INMET bigquery request

In [26]:
inmet_db = 'datario.meio_ambiente_clima.meteorologia_inmet'

inmet_live_query = groupby_first_query(inmet_db, 'id_estacao', 'data_particao', 'horario', select=['*'], max_delay=300)
query_job = client.query(inmet_live_query)
inmet_live = pd.DataFrame(list(map(dict, query_job.result())))

# data type conversion
inmet_live[['data_particao', 'horario']] = inmet_live[['data_particao', 'horario']].astype(str)

inmet_live.head()

,primary_key,id_estacao,pressao,pressao_minima,pressao_maxima,temperatura_orvalho,temperatura_orvalho_minimo,temperatura_orvalho_maximo,umidade,umidade_minima,...,temperatura_minima,temperatura_maxima,rajada_vento_max,direcao_vento,velocidade_vento,radiacao_global,acumulado_chuva_1_h,horario,data_particao,row_number
0,A636_2022-12-21 23:00:00,A636,1010.8,1010.8,1011.1,19.8,19.7,20.3,89.0,89.0,...,21.4,22.1,6.2,233,1.3,2.5,1.2,23:00:00,2022-12-21,1
1,A621_2022-12-21 23:00:00,A621,1009.9,1009.9,1010.1,20.4,20.4,20.6,97.0,97.0,...,20.9,21.1,1.6,50,0.0,0.0,2.2,23:00:00,2022-12-21,1
2,A602_2022-12-21 23:00:00,A602,1011.8,1011.8,1011.9,-0.6,-4.0,3.4,24.0,18.0,...,20.4,20.7,4.3,184,2.1,0.8,0.0,23:00:00,2022-12-21,1
3,A652_2022-12-21 23:00:00,A652,1008.7,1008.7,1009.0,NaN,NaN,NaN,NaN,NaN,...,20.9,21.4,4.5,281,1.7,-1.5,0.4,23:00:00,2022-12-21,1


---
## Flat stations' observations

In [165]:
# Flat station observations function

row_map = lambda row: row[1].add_suffix(' - ' + row[0])

def flat_observations(data):
    return pd.concat(list(map(row_map, data.iterrows())))

#### INMET flat

In [168]:
key_cols = ['primary_key', 'data_particao', 'horario'], # row_number

inmet_by_station = inmet_live.set_index('id_estacao').drop(key_cols, axis=1)

inmet_flat = flat_observations(inmet_by_station)

inmet_flat

pressao - A621                       1009.9
pressao_minima - A621                1009.9
pressao_maxima - A621                1010.1
temperatura_orvalho - A621             20.4
temperatura_orvalho_minimo - A621      20.4
                                      ...  
direcao_vento - A602                  184.0
velocidade_vento - A602                 2.1
radiacao_global - A602                  0.8
acumulado_chuva_1_h - A602              0.0
row_number - A602                       1.0
Length: 72, dtype: float64

#### Alerta-Rio flat data

In [169]:
readings = [
    'acumulado_chuva_15_min',
    'acumulado_chuva_1_h',
    'acumulado_chuva_4_h',
    'acumulado_chuva_24_h',
    'acumulado_chuva_96_h'
]

alertario_by_station = alerts.set_index('id')[readings]

alertario_flat = flat_observations(alertario_by_station)

alertario_flat

acumulado_chuva_15_min - 1      0.0
acumulado_chuva_1_h - 1         0.0
acumulado_chuva_4_h - 1         0.0
acumulado_chuva_24_h - 1        0.0
acumulado_chuva_96_h - 1        7.2
                               ... 
acumulado_chuva_15_min - 33     0.0
acumulado_chuva_1_h - 33        0.0
acumulado_chuva_4_h - 33        0.0
acumulado_chuva_24_h - 33       0.4
acumulado_chuva_96_h - 33      15.6
Length: 165, dtype: float64

---
## Join coordinates

#### Load stations' coordinates

In [27]:
inmet_stations = pd.read_csv('../Dados/Estações/Estações_Meteorológicas_INMET.csv')
alertario_stations = pd.read_csv('../Dados/Estações/Estações_Pluviométricas_AlertaRio.csv')

alertario_stations['id_estacao'] = alertario_stations['id_estacao'].astype(str)

inmet_coords = inmet_stations.set_index('id_estacao')[['estacao', 'latitude', 'longitude']]
alertario_coords = alertario_stations.set_index('id_estacao')[['estacao', 'latitude', 'longitude']]

#### Join Alerta-Rio coordinates

In [17]:
alertas = alertario_live.join(alertario_coords, on='id_estacao')

alertas.head()

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,row_number,latitude,longitude
0,21_2022-12-26 03:00:00,21,0.0,0.0,0.0,0.0,14.2,03:00:00,2022-12-26,1,-22.92556,-43.31583
1,29_2022-12-26 03:00:00,29,0.0,0.0,0.0,0.0,8.0,03:00:00,2022-12-26,1,-22.85694,-43.54111
2,13_2022-12-26 03:00:00,13,0.0,0.0,0.0,0.0,10.0,03:00:00,2022-12-26,1,-22.89182,-43.31005
3,22_2022-12-26 03:00:00,22,0.0,0.0,0.0,0.0,5.2,03:00:00,2022-12-26,1,-22.90944,-43.68444
4,23_2022-12-26 03:00:00,23,0.0,0.0,0.0,0.0,2.2,03:00:00,2022-12-26,1,-22.89056,-43.27806


#### Join INMET coordinates

In [224]:
inmet_live.join(inmet_coords, on='id_estacao')

,primary_key,id_estacao,pressao,pressao_minima,pressao_maxima,temperatura_orvalho,temperatura_orvalho_minimo,temperatura_orvalho_maximo,umidade,umidade_minima,...,rajada_vento_max,direcao_vento,velocidade_vento,radiacao_global,acumulado_chuva_1_h,horario,data_particao,row_number,latitude,longitude
0,A602_2022-12-21 23:00:00,A602,1011.8,1011.8,1011.9,-0.6,-4.0,3.4,24.0,18.0,...,4.3,184,2.1,0.8,0.0,23:00:00,2022-12-21,1,-23.050278,-43.595556
1,A621_2022-12-21 23:00:00,A621,1009.9,1009.9,1010.1,20.4,20.4,20.6,97.0,97.0,...,1.6,50,0.0,0.0,2.2,23:00:00,2022-12-21,1,-22.861389,-43.411389
2,A636_2022-12-21 23:00:00,A636,1010.8,1010.8,1011.1,19.8,19.7,20.3,89.0,89.0,...,6.2,233,1.3,2.5,1.2,23:00:00,2022-12-21,1,-22.940000,-43.402778
3,A652_2022-12-21 23:00:00,A652,1008.7,1008.7,1009.0,NaN,NaN,NaN,NaN,NaN,...,4.5,281,1.7,-1.5,0.4,23:00:00,2022-12-21,1,-22.988333,-43.190556
